# Sample Data Wrangling Project

## OpenStreetMap Sample Project - Data Wrangling with MongoDB

In [ ]:
import pprint
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from IPython.display import Image, display

### 1 - Data Preparation

**Map Area: Saint-Joseph - Île de La Réunion**
- http://www.openstreetmap.org/relation/1282272#map=12/-21.2918/55.6440
- http://overpass-api.de/api/map?bbox=55.4871,-21.4039,55.8009,-21.1796

In [ ]:
Image(filename='LaReunion.png', width=300, height=300)

In [ ]:
import OpenStreetMapTools

In [ ]:
OSM_FILE = "Saint-Joseph.La-Reunion.osm" 
SAMPLE_FILE = "Saint-Joseph.La-Reunion.sample.osm"

#### 1.1 - Sample Data

In [ ]:
OpenStreetMapTools.process_sample(file_in=OSM_FILE, file_out=SAMPLE_FILE, k=K)

#### 1.1.1 - node and way xml tags

**Data Source:** https://www.data.gouv.fr/s/resources/fichier-fantoir-des-voies-et-lieux-dits/community/20150512-103719/Descriptif_FANTOIR.pdf

From FANTOIR database we extract all possible well spelled Way Types

In [ ]:
FANTOIR1016_WAY_TYPE = pd.read_csv("FANTOIR2016-WAY-TYPE.csv", sep=";")
expected_way_types = FANTOIR1016_WAY_TYPE.LIBELLE.apply(lambda x: x.title()).values
FANTOIR1016_WAY_TYPE.head()

In [ ]:
st_types = OpenStreetMapTools.audit(SAMPLE_FILE, expected_way_types)
pprint.pprint(dict(st_types))

From the sample data above, we can identify at least the following problems:
1. Street types are wrongling spelled or abbreviated
2. No street types
3. Mixed encoding type
4. Street name case not consistent (Full upercase or lowercase or mix)

##### A first cleansing approach to address the problems listed above...

In [ ]:
# Map to the correct Street type 
street_type_mapping = { 
    "ALLEE": "Allee",
    "Allee": "Allee",
    "Alle": "Allee",
    u'All\xe8e': "Allee",
    u'All\xe9e': "Allee",
    "avenue": "Avenue",
    "Chemain": "Chemin",
    "Ch.": "Chemin",
    "Ch ": "Chemin",
    "Ch": "Chemin",
    "Che": "Chemin",
    "Cemin": "Chemin",
    "chemin": "Chemin",
    "Ch.Lolotte": "Chemin Lolotte",
    "Imp": "Impasse",
    "Imp.": "Impasse",
    "IMPASSE": "Impasse",
    "impasse": "Impasse",
    "IMPdes": "Impasse des",
    "RUE": "Rue",
    "Voi": "Voie"
}

# Update wrongly spelled or abbreviated Street names 
# and complete if necesseray (i.e.: ""Katia et Maurice Kraft": "Rue Katia et Maurice Kraft")
street_name_mapping = {
    "Jean Lauret": "Lieu-dit Jean Lauret",
    "Fortune Hoarau": u"Rue Fortuné Hoarau",
    "Sentier Prunes": "Sentier Des Prunes",
    u"allée des flamant": "Allee Des Flamants",
    "Che De L'Ariege": "Chemin De L'Ariege",
    "Katia et Maurice Kraft": "Rue Katia et Maurice Kraft",
    "Grande Savanne": "Lieu-dit Grande Savanne",
    "Hubert Delisle": "Rue Hubert Delisle",
    "Ch Des Gueri Vit": "Impasse Guerit Vite",
    "Voi cite les saphirs": "Voie Cite les saphirs"
}

# Choice is made to avoid latin characters at this level of cleansing...

In [ ]:
for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = OpenStreetMapTools.update_name(name, street_type_mapping, street_name_mapping)
        print name, "=>", better_name

#### 1.2 - Full Dataset

#### 1.2.1 - node and way xml tags

In [ ]:
full_st_types = OpenStreetMapTools.audit(OSM_FILE, expected_way_types)
pprint.pprint(dict(full_st_types))

In [ ]:
for full_st_types, ways in st_types.iteritems():
    for name in ways:
        better_name = OpenStreetMapTools.update_name(name, street_type_mapping, street_name_mapping)
        print name, "=>", better_name

The cleansing developed on the sample dataset seems to address the main issues present in the full dataset

### 2 - From XML to JSON to MongoDB

#### 2.1 - Create JSON file

Based on the cleansing identified in previous section, Street names will be partially updated during the JSON file generation

In [ ]:
ways_json_data = OpenStreetMapTools.process_map(OSM_FILE, street_type_mapping, street_name_mapping, True)

In [ ]:
[x for x in ways_json_data if x["id"] == "3480487005"]

#### 2.2 - MongoDB Import

In [ ]:
!mongo OpenStreetMap --eval "db.dropDatabase()"

In [ ]:
!mongoimport -d OpenStreetMap -c LaReunion --file data/Saint-Joseph.La-Reunion.osm.json

#### 2.3 - Data Overview

In [ ]:
from pymongo import MongoClient
from bson.son import SON

client = MongoClient()

def pretty(documents):
    for document in documents:
        pprint.pprint(document)

In [ ]:
db = client.OpenStreetMap

Just peek the previously selected sample example from our python cleansing code

In [ ]:
pretty(db.LaReunion.find({"id": "3480487005"}))

Check that we have the same count as the result of our python cleansing code 

In [ ]:
db.LaReunion.count() == len(ways_json_data)

Count the number of "xml node" imported

In [ ]:
pipeline = [
    {"$unwind": "$type"},
    {"$group": {"_id": "$type", "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1), ("_id", -1)])}
]
nodes = pd.DataFrame(list(db.LaReunion.aggregate(pipeline)))
nodes.columns = ["node", "count"]
nodes.head()

How many different users have contributed to this database?

In [ ]:
len(db.LaReunion.find().distinct("created.user"))

When did the contribution take place?

In [ ]:
pipeline = [
    {"$project": {"_id": False, "timestamp":  "$created.timestamp", "user": "$created.user" } },
    {"$group": {"_id": "$timestamp",  "users": { "$sum": 1 }}}
]
contributions = pd.DataFrame(list(db.LaReunion.aggregate(pipeline)))
contributions["_id"] = pd.to_datetime(contributions._id)
contributions.columns = ["date", "users"]
axes = contributions.set_index(['date']).plot(figsize=(12,6), title="Number of users contribution by date")

We have quite recent contributions for this dataset, let's identify the top 10 contributors?

In [ ]:
pipeline = [
    { "$project": { "_id": False, "user": "$created.user" } },
    { "$group": { "_id": "$user", "count": { "$sum": 1 } } },
    { "$sort": SON([("count", -1), ("_id", -1)]) },
    { "$limit": 10 }
]
pretty(list(db.LaReunion.aggregate(pipeline)))

We can consider having 1 top contributor followed by 5 high contributors... let see the distribution of the contribution for all users

In [ ]:
pipeline = [
    { "$project": { "_id": False, "user": "$created.user" } },
    { "$group": { "_id": "$user", "count": { "$sum": 1 } } },
    { "$sort": SON([("count", -1), ("_id", -1)]) },
    { "$project": { "_id": "$count"} }
]
contributions = pd.DataFrame(list(db.LaReunion.aggregate(pipeline)))
contributions.describe()

The above ouput clearly show that we have outliers in term of user contribution... just see if mongodb can provide us the distribution of the all users contribution with a predefined bins.

In [ ]:
pipeline = [
    { "$project": { "_id": False, "user": "$created.user" } },
    { "$group": { "_id": "$user", "count": { "$sum": 1 } } },
    {
        "$bucket": {
            "groupBy": "$count",
            "boundaries": [ 1, 100, 10000, 25000, 100000 ],
            "default": "TOP",
            "output": {
              "count": { "$sum": 1 },
              "users": { "$push": "$_id" }
            }
        }
    }
]
contributions = pd.DataFrame(list(db.LaReunion.aggregate(pipeline)))
contributions

The result confirm that most user are below 100 contributions and the main contributors are composed of 5 high contributors  and TOP contributor

### 3 - How MongoDb will ease the data cleansing process...?

First focus on all document with type='node' and having a subdocument 'address'

In [ ]:
pipeline = [
    { "$match": { "type": "node", "address": { "$exists": True } } },
    { "$project": { "_id": False, 
                   "city": "$address.city", 
                   "housenumber": "$address.housenumber", 
                   "postcode": "$address.postcode", 
                   "street": "$address.street" } },
    #{ "$limit": 10 }
]
addresses = pd.DataFrame(list(db.LaReunion.aggregate(pipeline)))
addresses.shape

In [ ]:
addresses.describe()

There is some missing data for all selected attributes

In [ ]:
addresses.head()

#### 3.1 - City

Let see if we can identify any problems with city attribute...  Can we recover the missing data for the city? 

This will be possible if the postcode is not null... to crosscheck with the postcal code as available at "http://datanova.legroupe.laposte.fr/explore/dataset/laposte_hexasmal/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true"

Content is described at "https://www.data.gouv.fr/fr/datasets/base-officielle-des-codes-postaux/", we will the following fields Nom_commune	and Code_postal

In [ ]:
import io
import requests
url="http://datanova.legroupe.laposte.fr/explore/dataset/laposte_hexasmal/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true"
postalcodes=pd.read_csv(io.StringIO(requests.get(url).content.decode('utf-8')), sep=";")
postalcodes.head()

In [ ]:
codes = dict(zip(postalcodes.Code_postal, postalcodes.Nom_commune))

In [ ]:
citiesFromPostCode = addresses[(addresses.city.isnull() == True) & (addresses.postcode.isnull() == False)]
citiesFromPostCode.loc[:, ("city")] = citiesFromPostCode.postcode.apply(lambda x: codes[int(x)].title())
citiesFromPostCode

In [ ]:
addresses.city.unique()

Two additional name to be clean "u'Petite-\xcele'" and "u'Petit \xeele'"

Not a big deal... cities in our dataset with some capitalization issues, wrong spelling and one dash missing in name, we should have the following cities spelled as follows: __Le Tampon, Saint-Pierre, Saint-Joseph, Petite-Ile...__. 'nan' value could not be updated

We can quickly solve this with python calling title() and replacing space with a dash for some cities....

In [ ]:
def cleanName(x):
    if x == u'Petite-\xcele' or x == u'Petit \xeele':
        return u'Petite-Ile'
    elif x.startswith("Saint") or x.startswith("Petite"):
        return x.replace(" ", "-").title()
    elif x == "Petit" or x.startswith("Petit ") or x.startswith("PETIT"):
        return "Petite-Ile"
    elif x in ["Le Tampion", "Tampon"]:
        return "Le Tampon"
    elif x.startswith("St"):
        return x.replace("St", "Saint").title()
    else:
        return x.title()

In [ ]:
addresses.loc[:, ("city")] = addresses.city.dropna().apply(lambda x: str(cleanName(x)))
addresses.city.dropna().unique()

In [ ]:
rev_codes = {v: k for k, v in codes.items()}
def reviewCityNameForLookup(x):
    return rev_codes[x.upper().replace("-", " ").replace("SAINT", "ST")]

In [ ]:
citiesToPostCode = addresses[(addresses.city.isnull() == False) & (addresses.postcode.isnull() == True)]

citiesToPostCode.loc[:, ("postcode")] = citiesToPostCode.city.dropna().apply(lambda x: reviewCityNameForLookup(x))
citiesToPostCode

In [ ]:
len(addresses.postcode.dropna().unique()) == len(addresses.city.dropna().unique())

The above code addressed some issues identified in the City and PostCode fields, what about street? What are the issues?

We are going to use the same FANTOIR DATABASE as referenced above.... 

In [ ]:
FANTOIR1016 = pd.read_table("FANTOIR1016", header=None)
FANTOIR1016 = FANTOIR1016[FANTOIR1016[0].str.startswith("974") == True]
FANTOIR1016.head()

For this project we are only considering the first 41 characters in FANTOIR1016 database (see §3.4 of referenced document)

| Code département | Code direction | Code commune | Identifiant de la voie dans la commune | Clé RIVOLI | Code nature de voie | Libellé voie |

Next we are filtering all lines not having at least 11 characters before a first space
Last steps: split the data into two groupes
- | Code département | Code direction | Code commune | Identifiant de la voie dans la commune | Clé RIVOLI
- | Code nature de voie | Libellé voie |

In [ ]:
FANTOIR1016_974 = FANTOIR1016[0].apply(lambda x : pd.Series(x[:41]))
FANTOIR1016_974["KEEP"] = FANTOIR1016_974[0].apply(lambda x: len(x.split(' ')[0]) >= 11)
FANTOIR1016_974 = FANTOIR1016_974[FANTOIR1016_974.KEEP == True][0].apply(lambda x: pd.Series([x[:11].strip(), 
                                                                                              x[11:15].strip(), 
                                                                                              x[15:41].strip()]))
FANTOIR1016_974.columns = ["REFERENCE", "CODE", "VOIE"]
FANTOIR1016_974.head()


In [ ]:
FANTOIR1016_974_WAYS = pd.merge(left=FANTOIR1016_974, right=FANTOIR1016_WAY_TYPE, on="CODE")[["REFERENCE",
                                                                                              "CODE", 
                                                                                              "LIBELLE", 
                                                                                              "VOIE"]]

In [ ]:
FANTOIR1016_974_WAYS["WAY"] = FANTOIR1016_974_WAYS[["LIBELLE", "VOIE"]].apply(lambda x: ' '.join(x), axis=1)
FANTOIR1016_974_WAYS.head()

Now we have a clean and official list of street name for our region, let identify which street name are not in listed in this reference (street name will be uppercase and with not latin letter for comparison)

In [ ]:
xtd = {ord(u'’'): u"'", ord(u'é'): u'e', ord(u'è'): u'e', ord(u'É'): u'E',}
def tr(x):
    return x.translate(xtd).upper()

In [ ]:
addresses.loc[:, ("street")] = addresses.street.dropna().apply(lambda x: tr(x))
addresses.head()

In [ ]:
addresses["CHECKED"] = addresses.street.dropna().apply(lambda x: x in FANTOIR1016_974_WAYS.WAY.values)
addresses.head()

In [ ]:
addresses[addresses.CHECKED == False].street.head()

In [ ]:
addresses[addresses.CHECKED == False].count()

Let's see what could be the problems...

In [ ]:
FANTOIR1016_974_WAYS[FANTOIR1016_974_WAYS.VOIE.str.find("MARTINEL") != -1]

Two possibilities... cannot be solved at this level

In [ ]:
FANTOIR1016_974_WAYS[FANTOIR1016_974_WAYS.VOIE.str.find("CHARRIE") != -1]

Bad spelling...

In [ ]:
FANTOIR1016_974_WAYS[FANTOIR1016_974_WAYS.WAY.str.find("AVENUE DU GEN") != -1]

We also have some shortcut... to simplify the update, all names not referenced will be manually updated from a file

In [ ]:
streets = pd.DataFrame(addresses[addresses.CHECKED == False].street.unique())
streets["New"] = streets[0]
streets.columns = ["Old", "New"]
streets.to_csv("street_name_update.csv", 
               encoding='utf-8',
               index=False,
               quoting=2
              )

In [ ]:
# Read back the updated file (not completly updated... to show case only!!)
streets = pd.read_csv("street_name_update.csv")
streets.head()

In [ ]:
streetNames = dict(zip(streets.Old, streets.New))
def streetNameUpdate(name):
    return streetNames[name] if name in streetNames.keys() else name

In [ ]:
addresses.street.apply(lambda x: streetNameUpdate(x)).head()

We have identify some rules to (partially) update the addresses, why not updating the mongo database...

** Postal Code and City **

In [ ]:
# Make sure we have no space in postcode (should have enforced as integer...)
for n in db.LaReunion.find({ "type": "node", 
                            "address": { "$exists": True }, 
                            "address.postcode": { "$exists": True } }):
    postcode = n["address"]["postcode"].replace(' ', '')
    db.LaReunion.update_one({ "_id": n["_id"] }, { "$set": { "address.postcode":  postcode } }, upsert=False)

In [ ]:
for n in db.LaReunion.find({ "type": "node", 
                            "address": { "$exists": True }, 
                            "address.postcode": { "$exists": True } }):
    city = codes[int(n["address"]["postcode"])]
    db.LaReunion.update_one({ "_id": n["_id"] }, { "$set": { "address.city":  city } }, upsert=False)

In [ ]:
pipeline = [
    { "$match": { "type": "node", "address": { "$exists": True }, "address.postcode": { "$exists": True } } },
    { "$project": { "_id": False, 
                   "city": "$address.city", 
                   "housenumber": "$address.housenumber", 
                   "postcode": "$address.postcode", 
                   "street": "$address.street" } },
    { "$limit": 10 }
]
pd.DataFrame(list(db.LaReunion.aggregate(pipeline)))

In [ ]:
for n in db.LaReunion.find({ 
        "type": "node", 
        "address": { "$exists": True }, 
        "address.city": { "$exists": True },
        "address.postcode": { "$exists": False }}):
    postcode = reviewCityNameForLookup(cleanName(n["address"]["city"]))
    db.LaReunion.update_one({ "_id": n["_id"] }, { "$set": { "address.postcode":  postcode } }, upsert=False)

In [ ]:
pretty(list(db.LaReunion.find({ 
        "type": "node", 
        "address": { "$exists": True }, 
        "address.city": { "$exists": False },
        "address.postcode": { "$exists": False }}).limit(2)))

Still some cleaning to be done... but not possible with the information we have in hands

### 3 - Conclusion

The dataset is far not complete and accurate. The different steps followed during the cleansing process has demonstrated that the dataset can be completed and made more accurate with some simple rules and correlation with some external databases. The process did not delivered a clean and accurate database as a lot a of specific rules need to be put in place as well as a better identification of external database that will make possible an automated process.

I think it will be interesting to have an automated validation of the data during the update of OpenStreetMap, such as crosscheck of city, postcode, street name from external and official database on per country basis (if it does exists).